In [3]:
pip install requests

     ---------------------------------------- 62.6/62.6 kB 3.3 MB/s eta 0:00:00
     ---------------------------------------- 61.5/61.5 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 123.9/123.9 kB ? eta 0:00:00
     ---------------------------------------- 96.9/96.9 kB ? eta 0:00:00
     ---------------------------------------- 158.3/158.3 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import requests

In [5]:
def getrecords(base_url, endpoint, headers=None, no_of_records=100, limit=30, offset=0):
  """Fetch the no_of_records from the API. Returns the fetched data in list

  Args:
      base_url (str): Base url of the API
      endpoint (str): Specific Endpoint of the API
      headers (dict, optional): headers of the API. Defaults to None.
      no_of_records (int, optional): number of records to fetch. Defaults to 100.

  Returns:
      final_data (list): Appends the data to list and returns it
  """

  final_data = []
  offset = 0
  while no_of_records!=0:
    if no_of_records<limit:
      response = requests.get(base_url+endpoint+f'?offset={offset}&limit={no_of_records}', headers=headers)
      response_data = response.json()
      final_data.extend(response_data['data'])
      return final_data

    response = requests.get(base_url+endpoint+f'?offset={offset}&limit={limit}', headers=headers)
    response_data = response.json()
    next_endpoint = response_data['next'] # /mentorskool/v1/sales?offset=101&limit=100
    params = next_endpoint.split("?")[1]  # offset=101&limit=100
    offset = params.split('&')[0].split('=')[1] # 101
    final_data.extend(response_data['data']) # len(final_data) = 100
    no_of_records = no_of_records - limit # 500 - 100 = 400

  return final_data

In [6]:
base_url = "https://zucwflxqsxrsmwseehqvjmnx2u0cdigp.lambda-url.ap-south-1.on.aws"
endpoint = "/mentorskool/v1/sales"
headers = {"access_token": "fe66583bfe5185048c66571293e0d358"}
data = getrecords(base_url=base_url,
                      endpoint=endpoint,
                      headers=headers,
                      no_of_records=500,
                      limit=100)

In [7]:
len(data)

500

In [9]:
pip install pandas


     --------------------------------------- 10.7/10.7 MB 59.4 MB/s eta 0:00:00
     ------------------------------------- 341.8/341.8 kB 20.7 MB/s eta 0:00:00
     --------------------------------------- 15.6/15.6 MB 34.4 MB/s eta 0:00:00
     ------------------------------------- 502.3/502.3 kB 30.8 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import pandas as pd

In [11]:
def jsonToDataframe(json_inp):
    '''
    Returns the sum of two decimal numbers in binary digits.

    Parameters:
        json_inp (python dict): response of the API in form of json

    Returns:
        df (Pandas dataframe): This is the dataframe that got generated by
        parsing the json response from the API
    '''

    df = pd.json_normalize(json_inp)
    cleaned_column_names = [x if(len(x.split('.')) == 1) else x.split('.')[-1] for x in df.columns]
    df.columns = cleaned_column_names

    return df

In [12]:
sales_df = jsonToDataframe(data)

In [13]:
sales_df[sales_df.duplicated()]

,id,sales_amt,qty,discount,profit_amt,order_id,ship_mode,order_status,order_purchase_date,order_approved_at,...,product_name,colors,category,sub_category,date_added,manufacturer,sizes,upc,weight,product_photos_qty


In [14]:
sales_df.replace("null", None, inplace=True)

In [15]:
def product_size(df, product_name):
    sizes_list = df[df["product_name"]==product_name]["sizes"].unique()[0]
    if not sizes_list:
        sizes = []
        count_of_sizes = len(sizes)
    else:
        sizes = [size.strip() for size in sizes_list.split(',')]
        count_of_sizes = len(sizes)
    return count_of_sizes

In [16]:
total_sizes = []
for index in range(len(sales_df)):
    product = sales_df.iloc[index]["product_name"]
    sizes = product_size(sales_df, product)
    total_sizes.append(sizes)


sales_df["no_of_sizes"] = total_sizes
sales_df.head()

,id,sales_amt,qty,discount,profit_amt,order_id,ship_mode,order_status,order_purchase_date,order_approved_at,...,colors,category,sub_category,date_added,manufacturer,sizes,upc,weight,product_photos_qty,no_of_sizes
0,4102,2.0000,0,72.948,None,US-2017-102288,Standard Class,delivered,2017-11-24 15:58:00,2017-11-28 03:18,...,Blue,Office Supplies,Appliances,2017-01-17,None,7,826000000000,None,0,1
1,2698,261.9600,2,0.000,41.9136,CA-2014-145317,Standard Class,delivered,2018-07-11 19:46:00,2018-07-13 15:25,...,Pink,Furniture,Bookcases,2016-04-01,None,9,640000000000,None,4,1
2,6827,731.9400,3,0.000,219.582,CA-2016-118689,Standard Class,delivered,2018-05-15 20:24:00,2018-05-15 20:55,...,Pink,Furniture,Chairs,2016-11-04,None,"10,7,6,9,8",664000000000,None,2,5
3,8154,14.6200,2,0.000,6.8714,CA-2017-140151,First Class,delivered,2018-02-28 13:40:00,2018-03-01 13:35,...,Pink,Office Supplies,Labels,2016-08-01,None,11.5,887000000000,None,3,1
4,2624,957.5775,5,0.450,-383.031,CA-2017-127180,First Class,delivered,2018-01-07 21:45:00,2018-01-07 21:57,...,Blue,Furniture,Tables,2016-11-15,None,"10.5,10,8.5,8,13",888000000000,None,0,5


In [17]:
sales_df.loc[sales_df["no_of_sizes"].idxmax()]['product_name']

'Redi-Strip #10 Envelopes, 4 1/8 x 9 1/2'

In [18]:
sales_df[sales_df['product_name']=='Mitel 5320 IP Phone VoIP phone']['no_of_sizes']

8    0
Name: no_of_sizes, dtype: int64

In [19]:
sales_df["order_purchase_date"] = pd.to_datetime(sales_df["order_purchase_date"])
sales_df = sales_df.assign(month_name = lambda x: x["order_purchase_date"].dt.month_name())

In [20]:
highest_sales_df = sales_df.groupby(["month_name"], as_index = False)["sales_amt"].sum()
print(highest_sales_df.loc[highest_sales_df["sales_amt"].idxmax()])

month_name         May
sales_amt     20820.34
Name: 8, dtype: object


In [22]:
# Convert "profit_amt" column to numeric type
sales_df["profit_amt"] = pd.to_numeric(sales_df["profit_amt"], errors="coerce")

# Group by month and calculate sum of profit
highest_profit_df = sales_df.groupby(["month_name"], as_index=False)["profit_amt"].sum()

# Find the row with the highest profit
max_profit_row = highest_profit_df.loc[highest_profit_df["profit_amt"].idxmax()]

print(max_profit_row)


month_name        March
profit_amt    5129.2084
Name: 7, dtype: object


In [23]:
month_wise_data = sales_df.groupby("month_name").agg({"sales_amt": sum, "profit_amt": sum}).reset_index()

In [24]:
month_wise_data

,month_name,sales_amt,profit_amt
0,April,3878.7690,-56.0825
1,August,10406.1850,-350.6432
2,December,14170.7800,-617.9495
3,February,8219.5320,1329.5155
4,January,16846.8305,905.8689
5,July,9926.4818,-84.1150
6,June,17143.9808,-1297.0189
7,March,17618.2270,5129.2084
8,May,20820.3400,-832.6445
9,November,3314.8160,365.0812


In [25]:
# Calculate profit margin
month_wise_data['profit_margin'] = (month_wise_data['profit_amt'] / month_wise_data['sales_amt']) * 100

# Positive profit margin
month_wise_data[month_wise_data['profit_margin'] > 0].shape[0]

5

In [26]:
sales_df["delay"] = ((pd.to_datetime(sales_df["order_delivered_customer_date"]) - pd.to_datetime(sales_df["order_estimated_delivery_date"])) / pd.Timedelta(days=1)).fillna(0).astype(int)
sales_df.head()

,id,sales_amt,qty,discount,profit_amt,order_id,ship_mode,order_status,order_purchase_date,order_approved_at,...,sub_category,date_added,manufacturer,sizes,upc,weight,product_photos_qty,no_of_sizes,month_name,delay
0,4102,2.0000,0,72.948,NaN,US-2017-102288,Standard Class,delivered,2017-11-24 15:58:00,2017-11-28 03:18,...,Appliances,2017-01-17,None,7,826000000000,None,0,1,November,-5
1,2698,261.9600,2,0.000,41.9136,CA-2014-145317,Standard Class,delivered,2018-07-11 19:46:00,2018-07-13 15:25,...,Bookcases,2016-04-01,None,9,640000000000,None,4,1,July,-2
2,6827,731.9400,3,0.000,219.5820,CA-2016-118689,Standard Class,delivered,2018-05-15 20:24:00,2018-05-15 20:55,...,Chairs,2016-11-04,None,"10,7,6,9,8",664000000000,None,2,5,May,-13
3,8154,14.6200,2,0.000,6.8714,CA-2017-140151,First Class,delivered,2018-02-28 13:40:00,2018-03-01 13:35,...,Labels,2016-08-01,None,11.5,887000000000,None,3,1,February,6
4,2624,957.5775,5,0.450,-383.0310,CA-2017-127180,First Class,delivered,2018-01-07 21:45:00,2018-01-07 21:57,...,Tables,2016-11-15,None,"10.5,10,8.5,8,13",888000000000,None,0,5,January,-7


In [27]:
# Replace the delay with NaT, in the case where order_delivered_customer_date is NaT
sales_df['delay'] = sales_df['delay'].where(sales_df['order_delivered_customer_date'].notna(), pd.NaT)

In [28]:
# filter out the instances of NaT type
sales_df2 = sales_df[~pd.isna(sales_df['delay'])]

In [29]:
sales_df2["delay_status"] = sales_df2["delay"].apply(lambda x: "Late" if x>0 else "Early" if x<0 else "On-time" if x==0 else None)
sales_df2.head()

C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_3324\516371200.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_df2["delay_status"] = sales_df2["delay"].apply(lambda x: "Late" if x>0 else "Early" if x<0 else "On-time" if x==0 else None)


,id,sales_amt,qty,discount,profit_amt,order_id,ship_mode,order_status,order_purchase_date,order_approved_at,...,date_added,manufacturer,sizes,upc,weight,product_photos_qty,no_of_sizes,month_name,delay,delay_status
0,4102,2.0000,0,72.948,NaN,US-2017-102288,Standard Class,delivered,2017-11-24 15:58:00,2017-11-28 03:18,...,2017-01-17,None,7,826000000000,None,0,1,November,-5,Early
1,2698,261.9600,2,0.000,41.9136,CA-2014-145317,Standard Class,delivered,2018-07-11 19:46:00,2018-07-13 15:25,...,2016-04-01,None,9,640000000000,None,4,1,July,-2,Early
2,6827,731.9400,3,0.000,219.5820,CA-2016-118689,Standard Class,delivered,2018-05-15 20:24:00,2018-05-15 20:55,...,2016-11-04,None,"10,7,6,9,8",664000000000,None,2,5,May,-13,Early
3,8154,14.6200,2,0.000,6.8714,CA-2017-140151,First Class,delivered,2018-02-28 13:40:00,2018-03-01 13:35,...,2016-08-01,None,11.5,887000000000,None,3,1,February,6,Late
4,2624,957.5775,5,0.450,-383.0310,CA-2017-127180,First Class,delivered,2018-01-07 21:45:00,2018-01-07 21:57,...,2016-11-15,None,"10.5,10,8.5,8,13",888000000000,None,0,5,January,-7,Early


In [30]:
late_delivered = sales_df2[sales_df2["delay_status"]=="Late"]["order_id"].unique()
len(late_delivered)

28

In [31]:
sales_df2[sales_df2["delay_status"]=="Late"].groupby("VendorID")["order_id"].count()

VendorID
VEN01    3
VEN02    7
VEN03    8
VEN04    7
VEN05    4
Name: order_id, dtype: int64